## Импорт библиотек

In [1]:
import requests #Получение данных
import json 
import csv #Запись данных в файл
import pandas as pd
import numpy as np
import re
import os #работа с файлами, папками
import folium #для построения карты

## Парсинг

In [ ]:
f = open("api_key.txt", "r")
key = f.read()

In [ ]:
def coordinates_parser(df_houses,key):
    names_yandex = []
    lat = []
    long = []
    for i in list(df_houses.complex_name):
        name = i.replace('&','and')
        try:
            name = name.split('(')[1].split(')')[0]
        except:
            None
        url="https://search-maps.yandex.ru/v1/?text=жк %s&lang=ru_RU&ll=30.3077291, 59.936431&spn=0.7,0.70&results=500&apikey=%s"\
        % (name, key)

        r = requests.get(url)
        decoded_data=r.text.encode().decode('utf-8-sig')
        data = json.loads(decoded_data)
        feature_num = len(data['features'])

        it = 0
        mark = 'ACTIVE'

        while it<feature_num and mark =='ACTIVE':     
            if data['features'][it]['properties']['CompanyMetaData']['Categories'][0]['name'] == 'Жилой комплекс':
                names_yandex.append(data['features'][it]['properties']['name'])
                lat.append(data['features'][it]['geometry']['coordinates'][1])
                long.append(data['features'][it]['geometry']['coordinates'][0])
                mark = 'DONE'
            it=it+1

        if mark=='ACTIVE':
            names_yandex.append('None')
            lat.append('None')
            long.append('None')
        
    df_houses['name_yandex'] = names_yandex
    df_houses['longitude'] = long
    df_houses['latitude'] = lat
    
    return df_houses

In [ ]:
def infrastructure_parser(df_houses, key, word_for_searching):
    for g in list(zip(df_houses['longitude'],df_houses['latitude'])):

        url="https://search-maps.yandex.ru/v1/?text=%s&lang=ru_RU&ll=%f,%f&spn=0.03,0.020&results=500&apikey=%s" \
        %(word_for_searching, g[0],g[1],key)
        r = requests.get(url)
        decoded_data=r.text.encode().decode('utf-8-sig')
        data_js = json.loads(decoded_data)

        for i in range(len(data_js['features'])):
            longitude = data_js['features'][i]['geometry']['coordinates'][0]
            latitude = data_js['features'][i]['geometry']['coordinates'][1]
            name = data_js['features'][i]['properties']['CompanyMetaData']['name']
            address = data_js['features'][i]['properties']['CompanyMetaData']['address']
            com_id = data_js['features'][i]['properties']['CompanyMetaData']['id']
            class_count = len(data_js['features'][i]['properties']['CompanyMetaData']['Categories'])
            extra_categories = None
            try:
                main_class = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][0]['class']
                main_category = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][0]['name']

                if class_count>3:
                    class_count=3

                for k in range(1,class_count):
                    extra_categories = []
                    try:
                        category_name = data_js['features'][i]['properties']['CompanyMetaData']['Categories'][k]['name']
                        extra_categories.append(category_name)
                    except:
                        None
                with open(r"infrastructure/loaded_data %s.csv" %word_for_searching, "a", newline="", encoding='utf-8') as file: #'a' - запись строки
                    row = [com_id,longitude,latitude, name,address,main_class, main_category,extra_categories] #строка, которую будем записывать
                    writer = csv.writer(file)
                    writer.writerow(row)
            except:
                None

In [ ]:
top_categories = ['Аптека','Больница для взрослых','Супермаркет','Ресторан','Школа танцев','Кафе','Быстрое питание','Булочная, пекарня','Общеобразовательная школа','Торговый центр','Фитнес-клуб','Кофейня','Товары для дома',
                 'Магазин хозтоваров и бытовой химии','Булочная','Спортплощадка, воркаут','Магазин продуктов','Бар, паб','Зоомагазин',
                 'Детская больница','Магазин парфюмерии и косметики','Парк культуры и отдыха','Кондитерская','Пиццерия','Гимназия','Бассейн',
                 'Продуктовый гипермаркет','Лицей','Остановка общественного транспорта','Станция метро','Железнодорожный вокзал',
                 'Железнодорожная станция']

In [ ]:
def loading_and_combining_infr(df_houses, catefory_list,key):
    infr_all = pd.DataFrame(columns=['com_id', 'longitude', 'latitude', 'name', 'address', 'main_class',
       'main_category', 'extra_categories'])
    for i in catefory_list:
        infrastructure_parser(df_houses, key, i)
    files_infr = os.listdir('/infrastructure')
    for j in files_infr:
        infr = pd.read_csv('infrastructure/%s' %j,names=['com_id', 'longitude', 'latitude', 'name', 'address', 'main_class',
       'main_category', 'extra_categories'])
        infr_all = infr_all.append(infr)
    infr_all = infr_all.drop_duplicates(subset='com_id')
    return infr_all

## КХ

In [41]:
from clickhouse_driver import Client

In [42]:
client = Client('3.23.221.144', user='default',password='qwerty')

In [43]:
client.execute('SHOW DATABASES')

[('_temporary_and_external_tables',), ('default',), ('system',), ('test_db',)]

In [2]:
infr_for_loading = pd.read_csv('unique_infr_for_loading.csv') 
houses_with_coordinates = pd.read_csv('houses_cian_with_coordinate.csv') 

In [44]:
infr_for_loading.head()

,com_id,longitude,latitude,name,address,main_class,main_category,extra_categories
0,1.184434e+09,30.262339,59.967746,Fitness House,"Россия, Санкт-Петербург, Крестовский проспект, 24",fitness,Фитнес-клуб,['Бассейн']
1,1.295862e+09,30.189438,59.858693,Аптека для бережливых,"Россия, Санкт-Петербург, Ленинский проспект, 7...",drugstores,Аптека,NaN
2,3.417767e+10,30.345279,59.835383,Аптека Вита Экспресс,"Россия, Санкт-Петербург, улица Ленсовета, 70",drugstores,Аптека,NaN
3,5.626781e+10,30.141090,59.852993,Аптека для бережливых,"Россия, Санкт-Петербург, улица Адмирала Чероко...",drugstores,Аптека,NaN
4,9.424204e+09,30.165203,59.790398,Аптека для жизни,"Россия, Ленинградская область, Ломоносовский р...",drugstores,Аптека,NaN


In [45]:
houses_with_coordinates.head()

,complex_id,complex_name,name_yandex,longitude,latitude
0,96,Южная долина,ЖК Южная долина,30.642199,59.985329
1,44193,Орловский парк,Орловский парк,30.269371,60.056063
2,25135,Дом на Космонавтов,ЖК Дом на Космонавтов,30.352259,59.839722
3,14272,Цвета Радуги,ЖК Цвета Радуги,30.446569,60.069298
4,38206,Жемчужный берег,ЖК Жемчужный Берег,30.151388,59.857277


### Загрузка данных в КХ

### Отбор жк в радиусе 35км

In [ ]:
# До этого считали расстояние до ст.м Невский проспект (longitude = 30.326892, latitude = 59.935368, com_id = 241704174115)

### Гео кластеры

In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

In [ ]:
# используем houses_with_coordinates с жк в радиусе 35 км

In [9]:
cluster_geo = houses_with_coordinates.set_index('complex_id')[['longitude','latitude']]
standardisedX_geo = scale(cluster_geo)
standardisedX_geo = pd.DataFrame(standardisedX_geo, index=cluster_geo.index, columns=cluster_geo.columns)

In [10]:
model_geo30 = KMeans(n_clusters=30)
model_geo30.fit(standardisedX_geo)
predictions_geo30 = model_geo30.predict(standardisedX_geo)
cluster_geo['cluster30'] = predictions_geo30

In [11]:
cluster_geo.head()

,longitude,latitude,cluster30
complex_id,,,
96,30.642199,59.985329,6
44193,30.269371,60.056063,16
25135,30.352259,59.839722,24
14272,30.446569,60.069298,3
38206,30.151388,59.857277,17


### Расчет радиуса по кластерам

In [32]:
groups = cluster_geo.groupby('cluster30')['longitude'].count().reset_index(name='count').set_index('cluster30')

In [33]:
# Считаем координаты центра кластера
groups['cluster_lat'] = cluster_geo.groupby('cluster30')[['latitude']].min()+(cluster_geo.groupby('cluster30')[['latitude']].max()-cluster_geo.groupby('cluster30')[['latitude']].min())/2
groups['claster_long'] = cluster_geo.groupby('cluster30')[['longitude']].min() +(cluster_geo.groupby('cluster30')[['longitude']].max()-cluster_geo.groupby('cluster30')[['longitude']].min())/2


In [34]:
# Считаем радиус по долготе и широте
groups['longitude_r'] = (cluster_geo.groupby('cluster30')[['longitude']].max()-cluster_geo.groupby('cluster30')[['longitude']].min())/2
groups['latitude_r'] = (cluster_geo.groupby('cluster30')[['latitude']].max()-cluster_geo.groupby('cluster30')[['latitude']].min())/2

In [35]:
# Переводим в км, расчеты не точные, желательно тут использовать КХ
groups['latitude_km'] = groups['latitude_r']*111
groups['longitude_km'] = groups['longitude_r']*56

In [36]:
# строим по максимальному радиусу
groups['radius_max'] = groups.apply(lambda x: max(x['latitude_km'],x['longitude_km']),axis=1)

In [38]:
# Там, где в кластере 1 жк, радиус получается 0
groups.loc[groups.radius_max==0,'radius_max'] = 0.3

In [37]:
groups

,count,cluster_lat,claster_long,longitude_r,latitude_r,latitude_km,longitude_km,radius_max
cluster30,,,,,,,,
0,23,60.059900,30.199460,0.054805,0.057508,6.383443,3.069080,6.383443
1,2,55.461016,37.491619,0.049477,0.107516,11.934221,2.770712,11.934221
2,1,45.087429,39.045933,0.000000,0.000000,0.000000,0.000000,0.000000
3,54,60.074978,30.455490,0.057369,0.073227,8.128197,3.212664,8.128197
4,30,59.954960,30.413910,0.033760,0.030769,3.415359,1.890560,3.415359
5,1,59.372269,28.619692,0.000000,0.000000,0.000000,0.000000,0.000000
6,23,59.984612,30.674125,0.075614,0.059182,6.569147,4.234384,6.569147
7,2,58.728769,29.849392,0.003147,0.003439,0.381729,0.176232,0.381729
8,10,59.778208,30.069602,0.052379,0.070627,7.839597,2.933224,7.839597


### Карта с гео кластерами

In [39]:
color30 = ['lightgreen', 'gray', 'darkblue', 'darkpurple', 'darkgreen', 'cadetblue', 'black', 'darkred', 'beige', 'blue', 'purple', 'lightgray', 'orange', 'pink', 'lightred', 'red', 'lightblue', 'green','blue','lightgreen', 'gray', 'darkblue', 'darkpurple', 'darkgreen', 'cadetblue','red', 'lightblue', 'green','blue','lightgreen']

In [40]:
# Карта с гео кластерами
m = folium.Map(location=[59.936431,30.3077291], zoom_start = 10)
color = ['darkred','orange','green']

for i in groups.index:
    data_lat = groups.loc[i].cluster_lat
    data_long = groups.loc[i].claster_long
    rad = groups.loc[i].radius_max
    
    folium.Circle(radius=rad*1300,location=[data_lat,data_long], popup = i,color = color30[int(i)],fill=True,fill_color=color30[int(i)]).add_to(m)

for i in cluster_geo.cluster30.unique():
    data_lat = list(cluster_geo[cluster_geo.cluster30==i].latitude)
    data_long = list(cluster_geo[cluster_geo.cluster30==i].longitude)
    
    for j in range(len(data_lat)):
        folium.Marker(location=[data_lat[j],data_long[j]], popup = i,icon=folium.Icon(color = color30[i])).add_to(m)

# m.save("кластеры geo 30.html")

m

### Формирование переменных инфраструктуры (Запрос в КХ)

### Расчет суммарной инфраструктуры (PCA)

### Средняя инфраструктура по кластеру

### Категории развития инфраструктуры по кластерам (равномерно)

### Средняя цена за м2 для 1к квартир (В КХ, для предложений по выбранным жк)

### Ценовые кластеры (Гаусс) по жк

### Итоговая карта

In [ ]:
# Нужно подставить нужные датафреймы
m = folium.Map(location=[59.936431,30.3077291], zoom_start = 10)
color = ['darkred','orange','green']

#Кластеры
for i in groups.inf_lev.unique():
    data_lat = list(groups[groups.inf_lev==i].cluster_lat)
    data_long = list(groups[groups.inf_lev==i].claster_long)
    rad = list(groups[groups.inf_lev==i].radius_max)
    
    for j in range(len(data_lat)):
        folium.Circle(radius=rad[j]*1300,location=[data_lat[j],data_long[j]], popup = i,color = color[int(i)],fill=True,fill_color=color[int(i)]).add_to(m)

# Жк с уровнем цен
color2 = ['blue','green','lightgreen','orange','red','darkred']
for i in new_corr.gauss_price_level_new.unique():
    data_lat = list(new_corr[new_corr.gauss_price_level_new==i].latitude)
    data_long = list(new_corr[new_corr.gauss_price_level_new==i].longitude)
    
    for j in range(len(data_lat)):
        folium.Marker(location=[data_lat[j],data_long[j]], popup = i,icon=folium.Icon(color = color2[int(i)])).add_to(m)

        
        
# m.save("инфрастуктура кластеры gauss price.html")


m